# Create location lookup

> Fill in a module description here

In [1]:
#| default_exp create_location_lookup

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import warnings
import pandas as pd
from pandas.errors import DtypeWarning
from pathlib import Path
import json
import numpy as np
import time
import redis
from geopy.geocoders import Nominatim
from openai import OpenAI
import math
import geopy
import googlemaps
import os

from data_preprocessor import const

Load lookup table if it already exists

In [5]:
lon_lat_lookup_path = Path(const.output_path, "lon_lat_lookup.json")
if lon_lat_lookup_path.is_file():
    with open(lon_lat_lookup_path, "r") as f:
        place_lon_lat_lookup = json.loads(f.read())
else:
    place_lon_lat_lookup = {}

Create a lookup table of UK cities and towns to lon-lat

In [6]:
#|exports

df_worldcities = pd.read_csv(Path(const.data_path, 'worldcities.csv'))

# Filter to UK
df_worldcities = df_worldcities[df_worldcities['country'] == 'United Kingdom']

# Add cities

for i, row in df_worldcities.iterrows():
    place = row['city'].strip().lower()
    place_lon_lat_lookup[place] = { 'lat' : row['lat'], 'lng' : row['lng'] }
    
# Add administration areas, by averaging

admin_lon_lats = {}
for i, row in df_worldcities.iterrows():
    if pd.isna(row['admin_name']):
        continue

    if (row['city'].strip().lower() == row['admin_name'].strip().lower()) or \
        (row['admin_name'].strip().lower() == f"{row['city'].strip()}, City of".strip().lower()):
        place = row['city']
    else:
        place = f"{row['city'].strip()}, {row['admin_name'].strip()}".strip().lower()
    
    if place in place_lon_lat_lookup:
        continue
    if not place in admin_lon_lats:
        admin_lon_lats[place] = []
    admin_lon_lats[place].append( (row['lat'], row['lng']) )
    
# average the administration areas

for k,vs in admin_lon_lats.items():
    place_lon_lat_lookup[k] = {
        'lat' : sum(map(lambda e: e[0], vs)) / len(vs),
        'lng' : sum(map(lambda e: e[1], vs))  / len(vs),
    }

Convert locations in the scraped sponsor data to lon-lats as well

In [7]:
#|exports
df_sponsors = pd.read_csv(Path(const.data_path, "all-skilled-home-care-sponsors.csv"))
df_sponsors.head()

,Organisation Name,Town/City,County,Type & Rating,Route,First appeared,Last appeared,Status
0,(IECC Care) Independent Excel Care Consortium ...,Colchester,NaN,Worker (A rating),Skilled Worker,2023-06-15,2024-03-01,Active
1,1 Oak Home Care,Sudbury,Suffolk,Worker (A rating),Skilled Worker,2022-10-17,2024-03-01,Active
2,1st Class Care Services2 Ltd,Ledbury,NaN,Worker (A rating),Skilled Worker,2023-03-06,2024-03-01,Active
3,1st Enable Ltd,Ellesmere Port,Cheshire,Worker (A rating),Skilled Worker,2022-09-30,2024-03-01,Active
4,1st Homecare Solutions Limited,Leighton Buzzard,Bedfordshire,Worker (A rating),Skilled Worker,2022-08-09,2024-03-01,Active


In [8]:
#|exports
client = OpenAI()

def correct_placename(placename):
    system_context = (
        "Given a mispelled version of a UK placename, you output either of the following: "
        "If you think it is a mispelled version of a UK placename, you output the corrected version. "
        "If you think string does not refer to a place, you output 'INVALID'."
    )
   
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_context},
            {"role": "user", "content": placename},
        ]
    )
    
    return response.choices[0].message.content

In [9]:
correct_placename("Stonehouse, Glos")

'Stonehouse, Gloucestershire'

In [10]:
#|exports
places_corrections_path = Path(const.output_path, "places_corrections.json")
if places_corrections_path.is_file():
    with open(lon_lat_lookup_path, "r") as f:
        places_corrections = json.loads(f.read())
else:
    places_corrections = {}

In [11]:
#|exports
df_sponsors = pd.read_csv(Path(const.data_path, "all-skilled-home-care-sponsors.csv"))
df_sponsors['Town/City'] = df_sponsors['Town/City'].str.lower().str.strip()

places = []

for i, row in df_sponsors.iterrows():
    if pd.isna(row['County']):
        places.append(row['Town/City'].lower().strip())
    elif pd.isna(row['Town/City']):
        places.append(row['County'].lower().strip())
    else:
        places.append(f"{row['Town/City'].strip()}, {row['County'].strip()}".lower())
        
places = set(places)

In [14]:
#|exports
api_key = os.environ.get('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

places = places - set(place_lon_lat_lookup.keys())
places = places - set(places_corrections.keys())

for i, place in enumerate(places):
    print(f"{i+1}/{len(places)}", end='\r')
    
    success = False
    while not success:
        if place in place_lon_lat_lookup or place in places_corrections:
            continue

        result = gmaps.geocode(f"{place.title()}, UK")
        location = result[0]['geometry']['location']
        
        if not location:
            place_corrected = correct_placename(place).lower()
            result = gmaps.geocode(f"{place_corrected.title()}, UK")
            location = result[0]['geometry']['location']
            print(f"{place} -> {place_corrected}")
            if location: places_corrections[place] = place_corrected
        else:
            place_corrected = place
        
        if location:
            place_lon_lat_lookup[place_corrected] = { 'lat' : location['lat'], 'lng' : location['lng'] }
        else:
            print(f"  {place} still invalid")
            place_lon_lat_lookup[place_corrected] = "invalid"
            
        success = True

In [17]:
#|exports
"""
geolocator = Nominatim(user_agent="Alex Guli")

places = places - set(place_lon_lat_lookup.keys())
places = places - set(places_corrections.keys())

for i, place in enumerate(places):
    print(f"{i+1}/{len(places)}", end='\r')
    
    success = False
    while not success:
        try:
            if place in place_lon_lat_lookup or place in places_corrections:
                continue

            location = geolocator.geocode(f"{place.title()}, UK")
            
            if location is None:
                place_corrected = correct_placename(place).lower()
                location = geolocator.geocode(f"{place_corrected.title()}, UK")
                print(f"{place} -> {place_corrected}")
                if location is not None: places_corrections[place] = place_corrected
            else:
                place_corrected = place
            
            if location is not None:
                place_lon_lat_lookup[place_corrected] = { 'lat' : location.latitude, 'lng' : location.longitude }
            else:
                print(f"  {place} still invalid")
                place_lon_lat_lookup[place_corrected] = "invalid"
                
            success = True
        except geopy.exc.GeocoderUnavailable as e:
            print("\nFailed API")
            time.sleep(30)
"""

'\ngeolocator = Nominatim(user_agent="Alex Guli")\n\nplaces = places - set(place_lon_lat_lookup.keys())\nplaces = places - set(places_corrections.keys())\n\nfor i, place in enumerate(places):\n    print(f"{i+1}/{len(places)}", end=\'\r\')\n    \n    success = False\n    while not success:\n        try:\n            if place in place_lon_lat_lookup or place in places_corrections:\n                continue\n\n            location = geolocator.geocode(f"{place.title()}, UK")\n            \n            if location is None:\n                place_corrected = correct_placename(place).lower()\n                location = geolocator.geocode(f"{place_corrected.title()}, UK")\n                print(f"{place} -> {place_corrected}")\n                if location is not None: places_corrections[place] = place_corrected\n            else:\n                place_corrected = place\n            \n            if location is not None:\n                place_lon_lat_lookup[place_corrected] = { \'lat\' : loc

In [18]:
#|hide
# Will not use postcodes

"""

# Ignore DtypeWarning
warnings.filterwarnings('ignore', category=DtypeWarning)

# Add postcodes
r = redis.Redis()
pipe = r.pipeline()

path_dfs = list(Path(const.data_path, 'postcode_data').glob('*.csv'))
for i, path_df in enumerate(path_dfs):
    print(f"Processing postcode data ({i+1}/{len(path_dfs)})", end="\r")
    df_postcode = pd.read_csv(path_df)
    
    for j, row in df_postcode.iterrows():
        pipe.set(row['pcd'], f"{row['lat']}, {row['lng']}")
        place_lon_lat_lookup[row['pcd']] = { 'lat' : row['lat'], 'lng' : row['long'] }
        place_lon_lat_lookup[row['pcd2']] = { 'lat' : row['lat'], 'lng' : row['long'] }
        place_lon_lat_lookup[row['pcds']] = { 'lat' : row['lat'], 'lng' : row['long'] }

pipe.execute()
"""

'\n\n# Ignore DtypeWarning\nwarnings.filterwarnings(\'ignore\', category=DtypeWarning)\n\n# Add postcodes\nr = redis.Redis()\npipe = r.pipeline()\n\npath_dfs = list(Path(const.data_path, \'postcode_data\').glob(\'*.csv\'))\nfor i, path_df in enumerate(path_dfs):\n    print(f"Processing postcode data ({i+1}/{len(path_dfs)})", end="\r")\n    df_postcode = pd.read_csv(path_df)\n    \n    for j, row in df_postcode.iterrows():\n        pipe.set(row[\'pcd\'], f"{row[\'lat\']}, {row[\'lng\']}")\n        place_lon_lat_lookup[row[\'pcd\']] = { \'lat\' : row[\'lat\'], \'lng\' : row[\'long\'] }\n        place_lon_lat_lookup[row[\'pcd2\']] = { \'lat\' : row[\'lat\'], \'lng\' : row[\'long\'] }\n        place_lon_lat_lookup[row[\'pcds\']] = { \'lat\' : row[\'lat\'], \'lng\' : row[\'long\'] }\n\npipe.execute()\n'

Save lookup table

In [19]:
#|exports
with open(lon_lat_lookup_path, 'w') as f:
    json.dump(place_lon_lat_lookup, f)
    
with open(places_corrections_path, 'w') as f:
    json.dump(places_corrections, f)

In [20]:
#|exports
def lat_lng_to_mercator(lat, lng):
    r_major = 6378137.000
    x = r_major * math.radians(lng)
    scale = x/lng
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [21]:
#|exports
place_merc_lookup = { place : lat_lng_to_mercator(loc['lat'], loc['lng']) for place, loc in place_lon_lat_lookup.items() }    
place_merc_lookup_path = Path(const.output_path, "place_merc_lookup.json")
with open(place_merc_lookup_path, 'w') as f:
    json.dump(place_merc_lookup, f)

Update the sponsor data with location

In [22]:
#|exports
df_sponsors = pd.read_csv(Path(const.data_path, "current-skilled-home-care-sponsors.csv"))
df_sponsors['Town/City'] = df_sponsors['Town/City'].str.lower().str.strip()
df_sponsors['Town/City'] = df_sponsors['Town/City'].replace(places_corrections)
df_sponsors.head()

,Organisation Name,Town/City,County,Type & Rating,Route,First appeared,Last appeared,Status
0,(IECC Care) Independent Excel Care Consortium ...,colchester,NaN,Worker (A rating),Skilled Worker,2023-06-15,2024-03-01,Active
1,1 Oak Home Care,sudbury,Suffolk,Worker (A rating),Skilled Worker,2022-10-17,2024-03-01,Active
2,1st Class Care Services2 Ltd,ledbury,NaN,Worker (A rating),Skilled Worker,2023-03-06,2024-03-01,Active
3,1st Enable Ltd,ellesmere port,Cheshire,Worker (A rating),Skilled Worker,2022-09-30,2024-03-01,Active
4,1st Homecare Solutions Limited,leighton buzzard,Bedfordshire,Worker (A rating),Skilled Worker,2022-08-09,2024-03-01,Active


In [23]:
#|exports
xs, ys = [], []

for i, row in df_sponsors.iterrows():
    if pd.isna(row['County']):
        place = row['Town/City'].lower().strip()
    elif pd.isna(row['Town/City']):
        place = row['County'].lower().strip()
    else:
        place = f"{row['Town/City'].strip()}, {row['County'].strip()}".lower()
    
    if place in place_lon_lat_lookup:
        x, y = place_merc_lookup[place]
        xs.append(x)
        ys.append(y)
    else:
        print(place)
        xs.append(-1)
        ys.append(-1)

In [24]:
#|exports
df_sponsors['merc_x'] = xs
df_sponsors['merc_y'] = ys

df_sponsors.to_csv(Path(const.output_path, "current-skilled-home-care-sponsors-with-loc.csv"), index=False)